In [156]:
import os
import pickle
# Gmail API utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode
# for dealing with attachement MIME types
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from mimetypes import guess_type as guess_mime_type
import requests
import json
import simplejson
import re
import boto3
from datetime import date
from datetime import datetime, timedelta
import ipfsApi
import ipfshttpclient
import subprocess
import folium
import pandas as pd
from io import StringIO

In [177]:
date = datetime.utcnow().date()

In [ ]:
# Get route data
location_dictionary = {'Dublin': {'lat': '53.3470', 'lon': '-6.258394', 'country': 'Ireland'}, 
                       'Paris': {'lat': '48.849338', 'lon': '2.329523', 'country': 'France'},
                       'Berlin': {'lat': '52.506312', 'lon': '13.391077', 'country': 'Germany'},
                       'Madrid': {'lat': '40.432026', 'lon': '-3.687545', 'country': 'Spain'},
                       'London': {'lat': '51.506414', 'lon': '-0.110954', 'country': 'United Kingdom'}
                      }
incident_list = []
for location in location_dictionary:
    lat = location_dictionary.get(location).get('lat')
    lon = location_dictionary.get(location).get('lon')
    country = location_dictionary.get(location).get('country')
    location = location
    url = "https://data.traffic.hereapi.com/v7/incidents?in=circle:{},{};r=2000&locationReferencing=shape&lang=en-US&inc22=0&details=true&apiKey=BRu0LRXmYLe-Im_GKb8zBltxJjGVlXeWDS8LOCNPJLA".format(lat,lon)
    page = requests.get(url)
    json_string = page.json()
    for i in json_string['results']:
        incident_dict = {}
        incident_dict['country'] = country
        incident_dict['city'] = location
        incident = i['incidentDetails']['description']['value']
        try:
            incident_dict['lat'] = i['location']['shape']['links'][0]['points'][0]['lat']
            incident_dict['lon'] = i['location']['shape']['links'][0]['points'][0]['lng']

        except:
            print('no lat or long')
        incident_dict['incident'] = incident
        
        incident_list.append(incident_dict)
    

In [96]:
df_incidents = pd.DataFrame(incident_list)
df_incidents.to_csv('Traffic_incidents_' + str(date) + '.csv')

In [ ]:
# parse data
todays_data = dict()
for i in json_string['results']:
    loc = i['incidentDetails']['description']['value']

    if loc in todays_data:
        todays_data[loc] += 1
    else:
        todays_data[loc] = 1

In [ ]:
mymap = folium.Map()
for location in incident_list:
    folium.Marker(
    location=[location['lat'], location['lon']],
    popup=location['incident']).add_to(mymap)

In [ ]:
mymap.save("Traffic_incident_map.html")

In [ ]:
def create_alert(incident_list):
    alert_list = []
    for alert in incident_list:
        incident = alert['incident']
        if incident == 'Closed':
            incident = 'Road closed'
        if ' - ' in incident:
            incident_split = incident.split(' - ')
            incident = incident_split[1] + ' ' + incident_split[0][0].lower() + incident_split[0][1:]
        city = alert['city']
        country = alert['country']
        alert = incident + ' in ' + city + ', ' + country
        alert_list.append(alert)
    return alert_list
    

In [ ]:
alert_list = create_alert(incident_list)
df = pd.DataFrame (alert_list, columns = ['Traffic Incidents'])
result = df.to_html()
text_file = open("Traffic_Incidents.html", "w")
text_file.write(result)
text_file.close()

In [98]:
output = subprocess.check_output([r"C:\Program Files\IPFS Desktop\resources\app.asar.unpacked\node_modules\go-ipfs\go-ipfs\ipfs.exe","add",r"C:\Users\kearn\Desktop\Notebooks\Here traffic ipfs\Traffic_incidents_"+ str(date) + ".csv"])
output = output.decode('utf-8')
unique_hashcode = output.split(' ')[1]
output_file = subprocess.check_output([r"C:\Program Files\IPFS Desktop\resources\app.asar.unpacked\node_modules\go-ipfs\go-ipfs\ipfs.exe","cat",unique_hashcode])

In [99]:
output

'added QmRGZ1s1yReikWPwEiFk1zdZiFre5w9Z9znUB1sCD5Qe92 Traffic_incidents_2022-12-03.csv\n'

In [100]:
unique_hashcode

'QmRGZ1s1yReikWPwEiFk1zdZiFre5w9Z9znUB1sCD5Qe92'

In [147]:
with open("hashcodes.txt", "r") as myfile:
    contents = myfile.readlines()
    hashcodes = json.loads(contents[0])
    hashcodes[str(yesterday)] = unique_hashcode
    print(hashcodes)

{'2022-12-03': 'QmRGZ1s1yReikWPwEiFk1zdZiFre5w9Z9znUB1sCD5Qe92', '2022/12/02': 'QmRGZ1s1yReikWPwEiFk1zdZiFre5w9Z9znUB1sCD5Qe92'}


In [149]:
with open("hashcodes.txt", "w") as myfile:
    myfile.write(json.dumps(hashcodes))

In [121]:
# with open('hashcodes.txt', 'w') as convert_file:
#      convert_file.write(json.dumps(hashcodes))

In [101]:
output_file

b",country,city,lat,lon,incident\r\n0,Ireland,Dublin,53.36061003617942,-6.239010030403733,At Talbot Street - Road construction\r\n1,Ireland,Dublin,53.337970012798905,-6.255860002711415,At Merrion Row (N11) - Restrictions\r\n2,Ireland,Dublin,53.35500002838671,-6.229230025783181,At Church Road - Road construction\r\n3,Ireland,Dublin,53.35543002001941,-6.2421400006860495,Road construction\r\n4,Ireland,Dublin,53.33830000832677,-6.255630003288388,Closed\r\n5,Ireland,Dublin,53.359389966353774,-6.240320038050413,At Fairview Strand - Road construction\r\n6,Ireland,Dublin,53.35910003632307,-6.269259983673692,Between Berkeley Road and Upper Dorset Street (N1) - Road construction\r\n7,Ireland,Dublin,53.33791997283697,-6.276210006326437,Between Ward's Hill and Ardee Street - Closed\r\n8,Ireland,Dublin,53.35569002665579,-6.261259960010648,At Parnell Street - Restrictions\r\n9,Ireland,Dublin,53.35580997169018,-6.2609899789094925,At Great Denmark Street - Restrictions\r\n10,Ireland,Dublin,53.35698000

In [ ]:
our_email = 'kearns.dylan@gmail.com'
SCOPES = ['https://mail.google.com/']

In [ ]:
def gmail_authenticate():
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

In [ ]:
def add_attachment(message, filename):
    content_type, encoding = guess_mime_type(filename)
    if content_type is None or encoding is not None:
        content_type = 'application/octet-stream'
    main_type, sub_type = content_type.split('/', 1)
    print(main_type)
    if main_type == 'text':
        fp = open(filename, 'rb')
        msg = MIMEText(fp.read().decode(), _subtype=sub_type)
        fp.close()
    elif main_type == 'image':
        fp = open(filename, 'rb')
        msg = MIMEImage(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'audio':
        fp = open(filename, 'rb')
        msg = MIMEAudio(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'application':
        fp = open(filename, 'rb')
        msg = MIMEAudio(fp.read(), _subtype=sub_type)
        fp.close() 
    
    else:
        fp = open(filename, 'rb')
        msg = MIMEBase(main_type, sub_type)
        msg.set_payload(fp.read())
        fp.close()
    filename = os.path.basename(filename)
    msg.add_header('Content-Disposition', 'attachment', filename=filename)
    message.attach(msg)

In [ ]:
def build_message(destination, obj, body, attachments=[]):
    if not attachments: # no attachments given
        message = MIMEText(body)
        message['to'] = destination
        message['from'] = our_email
        message['subject'] = obj
    else:
        message = MIMEMultipart()
        message['to'] = destination
        message['from'] = our_email
        message['subject'] = obj
        message.attach(MIMEText(body))
        for filename in attachments:
            add_attachment(message, filename)
    return {'raw': urlsafe_b64encode(message.as_bytes()).decode()}

In [ ]:
def send_message(service, destination, obj, body, attachments=[]):
    return service.users().messages().send(
      userId="me",
      body=build_message(destination, obj, body, attachments)
    ).execute()

In [ ]:
# for email in emails:
send_message(service, "kearns.dylan@gmail.com", 'Traffic incidents ' + str(date), 
'Please find the traffic alerts for today' , ['Traffic_Incidents.html','Traffic_incident_map.html'])

In [106]:
yesterday = date - timedelta(days = 1)

In [117]:
yesterday = yesterday.strftime("%Y/%m/%d")

In [118]:
yesterday

'2022/12/02'

In [151]:
with open("hashcodes.txt", "r") as myfile:
    contents = myfile.readlines()
    hashcodes_two = json.loads(contents[0])

In [153]:
yesterday_hash = hashcodes_two.get(yesterday)
yesterday_output_file = subprocess.check_output([r"C:\Program Files\IPFS Desktop\resources\app.asar.unpacked\node_modules\go-ipfs\go-ipfs\ipfs.exe","cat",yesterday_hash])

In [157]:
s=str(yesterday_output_file,'utf-8')

data = StringIO(s) 

df_yesterday=pd.read_csv(data)

In [163]:
df_yesterday = df_yesterday.drop(columns = ['Unnamed: 0'])

In [166]:
sample = df_yesterday.head(5)

,country,city,lat,lon,incident
0,Ireland,Dublin,53.36061,-6.23901,At Talbot Street - Road construction
1,Ireland,Dublin,53.33797,-6.25586,At Merrion Row (N11) - Restrictions
2,Ireland,Dublin,53.35500,-6.22923,At Church Road - Road construction
3,Ireland,Dublin,53.35543,-6.24214,Road construction
4,Ireland,Dublin,53.33830,-6.25563,Closed


In [173]:
intersection_df = pd.merge(sample, df_incidents, how ='inner', on =['lat', 'lon'])
intersection_df = intersection_df.drop(columns=['incident_y','country_y', 'city_y'])
intersection_df = intersection_df.rename(columns={"country_x": "country", "city_x": "city", "incident_x": "incident"})

In [175]:
# intersection_df.to_csv('Traffic_incidents_tracking.csv', index=False)

In [176]:
traffic_incidents_tracking_df = pd.read_csv('Traffic_incidents_tracking.csv')
traffic_incidents_tracking_intersection_df = pd.merge(traffic_incidents_tracking_df, intersection_df, how ='inner', on =['lat', 'lon'])
traffic_incidents_tracking_intersection_df = traffic_incidents_tracking_intersection_df.drop(columns=['incident_y','country_y', 'city_y'])
traffic_incidents_tracking_intersection_df = traffic_incidents_tracking_intersection_df.rename(columns={"country_x": "country", "city_x": "city", "incident_x": "incident"})

,country_x,city_x,lat,lon,incident_x,country_y,city_y,incident_y
0,Ireland,Dublin,53.36061,-6.23901,At Talbot Street - Road construction,Ireland,Dublin,At Talbot Street - Road construction
1,Ireland,Dublin,53.33797,-6.25586,At Merrion Row (N11) - Restrictions,Ireland,Dublin,At Merrion Row (N11) - Restrictions
2,Ireland,Dublin,53.35500,-6.22923,At Church Road - Road construction,Ireland,Dublin,At Church Road - Road construction
3,Ireland,Dublin,53.35543,-6.24214,Road construction,Ireland,Dublin,Road construction
4,Ireland,Dublin,53.33830,-6.25563,Closed,Ireland,Dublin,Closed


In [ ]:
traffic_incidents_tracking_intersection_df.to_csv('Traffic_incidents_tracking.csv', index=False)